In [9]:
import requests
from io import BytesIO
import zipfile
import xmltodict
import json
from bs4 import BeautifulSoup

DART_API_KEY = ""

## get all codes
# 참고: corp_code는 Dart에서 종목별로 부여한 고유번호로, 아래처럼 호출하면 모두 볼 수 있음(ref: http://blog.quantylab.com/2020-10-09-dart.html)

def get_corp_codes():
    api_url = 'https://opendart.fss.or.kr/api/corpCode.xml'
    res = requests.get(api_url, params={'crtfc_key': DART_API_KEY})
    z = zipfile.ZipFile(BytesIO(res.content))
    data_xml = z.read('CORPCODE.xml').decode('utf-8')
    data_odict = xmltodict.parse(data_xml)
    data_dict = json.loads(json.dumps(data_odict))
    data = data_dict.get('result', {}).get('list')
    return data
corp_codes = get_corp_codes()

## 만약 같은 이름의 회사 2개가 있을 경우 첫번째 값을 호출 (예를 들어 삼성물산은 2개가 있음)
def get_corp_code(corp_name):
    tmp = []
    for item in corp_codes:
        if item['corp_name'] == corp_name:
            tmp.append(item)
            # corp_code = item['corp_name']
            # print(item)
    return tmp[0]['corp_code']

### input : 회사명, output : [기재정정]투자설명서 문서번호
def get_inv_rcpNo(corp_name):
    corp_code = get_corp_code(corp_name)
    api_url = 'https://opendart.fss.or.kr/api/list.json'
    params = {
        'crtfc_key': DART_API_KEY,
        'corp_code': corp_code,
        'bgn_de' : 20160117

    }
    mylist = list(filter(lambda item: '[기재정정]투자설명서' in item['report_nm'] , requests.get(api_url, params=params).json()['list']))
    return mylist[0]['rcept_no']

### input : 회사명, output : 증권발행실적보고서 문서번호
def get_issueResult_rcpNo(corp_name):
    corp_code = get_corp_code(corp_name)
    api_url = 'https://opendart.fss.or.kr/api/list.json'
    params = {
        'crtfc_key': DART_API_KEY,
        'corp_code': corp_code,
        'bgn_de' : 20160117

    }
    mylist = list(filter(lambda item: '발행실적' in item['report_nm'] , requests.get(api_url, params=params).json()['list']))
    print(mylist)
    return mylist[0]['rcept_no']



# 코난테크놀로지 투자설명서
import re
from bs4 import BeautifulSoup
rcpNo = '20220624000086'

## 투자설명서 파싱
def parse_investings(rcpNo):

    url_doc = f'https://dart.fss.or.kr/dsaf001/main.do?rcpNo={rcpNo}'
    r = requests.get(url_doc).text


    target = re.search(r'"I. 모집 또는 매출에 관한 일반사항";((.|\n)*?)/', r, re.M).group(1)
    target = re.sub('\s+', '', target).split(";")
    dcmNo = target[2].split("=")[1].replace('"', '')
    eleId = target[3].split("=")[1].replace('"', '')
    offset = target[4].split("=")[1].replace('"', '')
    length = target[5].split("=")[1].replace('"', '')
    dtd = target[6].split("=")[1].replace('"', '')
    # 모집에 관한 일반사항
    final_url = f"https://dart.fss.or.kr/report/viewer.do?rcpNo={rcpNo}&dcmNo={dcmNo}&eleId={eleId}&offset={offset}&length={length}&dtd={dtd}"
    html = requests.get(final_url).text
    soup = BeautifulSoup(html, 'html.parser')
    tables = soup('table')

    ### 공모개요 table
    summary_table_1 = tables[1]
    summary_table_2= tables[2]
    summary_table_3= tables[3]


    table_1 = list(
        filter(lambda x: "모집총액" in x.text, tables))
    print(len(table_1))   ## 공모 상세

    table_2 = list(
        filter(lambda x: "국내기관투자자" in x.text, tables))
    print(len(table_2))   ## 수요예측참여내역

    table_3 = list(
        filter(lambda x: "밴드상단 초과" in x.text, tables))
    print(len(table_3))   ## 수요예측 신청가격분포

    table_4 = list(
        filter(lambda x: "확약기간" in x.text, tables))
    print(len(table_4))   ## 확약기간

# var node3 = {}; 와 cnt++; 사이

In [18]:
# 코난테크놀로지 투자설명서
import re
from bs4 import BeautifulSoup
from collections import OrderedDict

rcpNo = '20220624000086'
## url https://dart.fss.or.kr/dsaf001/main.do?rcpNo=20220624000086
## 투자설명서 파싱
def parse_investings(rcpNo):
    info = OrderedDict()
    
    url_doc = f'https://dart.fss.or.kr/dsaf001/main.do?rcpNo={rcpNo}'
    r = requests.get(url_doc).text
    target = re.search(r'"I. 모집 또는 매출에 관한 일반사항";((.|\n)*?)/', r, re.M).group(1)
    target = re.sub('\s+', '', target).split(";")
    dcmNo = target[2].split("=")[1].replace('"', '')
    eleId = target[3].split("=")[1].replace('"', '')
    offset = target[4].split("=")[1].replace('"', '')
    length = target[5].split("=")[1].replace('"', '')
    dtd = target[6].split("=")[1].replace('"', '')
    # 모집에 관한 일반사항
    final_url = f"https://dart.fss.or.kr/report/viewer.do?rcpNo={rcpNo}&dcmNo={dcmNo}&eleId={eleId}&offset={offset}&length={length}&dtd={dtd}"
    
    
    
    html = requests.get(final_url).text
    soup = BeautifulSoup(html, 'html.parser')
    tables = soup('table')

    ### 공모개요 table
    info['stock_type'] = tables[1].select('td')[0].text ## 증권의 종류 : 기명식보통주
    info['stock_qty'] = tables[1].select('td')[1].text ## 증권수량
    info['face_value'] = tables[1].select('td')[2].text ## 액면가액
    info['final_price'] = tables[1].select('td')[3].text ## 최종 공모가


    ## 공모 상세
    table_1 = list(
        filter(lambda x: "모집총액" in x.text, tables))[-2]
    
    info['employee_qty'] = re.sub('[^0-9]', '',table_1.select('tr')[1].select('td')[1].text) ## 우리사주조합 배정주
    info['normal_qty'] = re.sub('[^0-9]', '',table_1.select('tr')[2].select('td')[1].text) ## 일반청약자 배정주
    info['org_qty'] =  re.sub('[^0-9]', '', table_1.select('tr')[3].select('td')[1].text) ## 기관투자자 배정주
    
    print(info)

    ## 수요예측참여내역
    table_2 = list(
        filter(lambda x: "국내기관투자자" in x.text, tables))[0]
    print(len(table_2))   

    ## 수요예측 신청가격분포 ## 19년 이후 보고서는 중간값 밴드로 이용
    table_3 = list(
        filter(lambda x: "밴드상단 초과" in x.text, tables))
    print(len(table_3))   

    
    ## 의무보유확약기간
    table_4 = list(
        filter(lambda x: "확약기간" in x.text, tables))
    print(len(table_4))


    ## 인수수수료율 (SK바이오팜은 0.8%) 같은 페이지에도 있음


    ## 1단계 :  상장 후 총 발행물량 / 유통가능물량 (이건 임시임)
    

    ############### 증권발행보고서>청약 및 배정에 관한 사항에서 알 수 있는 사항

    ## 우리사주 청약율 (최초 배정vs 최종 배정)
    ## 기관투자자 의무보유확약기간별 (최종)
    ## 유통가능주식수 (최종, 로직으로 계산하면 될 듯)



# var node3 = {}; 와 cnt++; 사이
parse_investings(rcpNo)

OrderedDict([('stock_type', '기명식보통주'), ('stock_qty', '1,200,000'), ('face_value', '500'), ('final_price', '25,000'), ('employee_qty', '\n60,000주\n'), ('normal_qty', '\n300,000주\n'), ('org_qty', '\n840,000주\n')])
1
1
5


In [22]:
rcpNo2 = '20220630000860'
## url https://dart.fss.or.kr/dsaf001/main.do?rcpNo=20220624000086
## 증권발행보고서 파싱
def parse_issueReport(rcpNo):
    info = OrderedDict()
    url_doc = f'https://dart.fss.or.kr/dsaf001/main.do?rcpNo={rcpNo}'

    # 청약 및 배정에 관한 사항 페이지로 넘어가기

    r = requests.get(url_doc).text
    target = re.search(r'"Ⅱ. 청약 및 배정에 관한 사항";((.|\n)*?)/', r, re.M).group(1)
    target = re.sub('\s+', '', target).split(";")
    dcmNo = target[2].split("=")[1].replace('"', '')
    eleId = target[3].split("=")[1].replace('"', '')
    offset = target[4].split("=")[1].replace('"', '')
    length = target[5].split("=")[1].replace('"', '')
    dtd = target[6].split("=")[1].replace('"', '')

    final_url = f"https://dart.fss.or.kr/report/viewer.do?rcpNo={rcpNo}&dcmNo={dcmNo}&eleId={eleId}&offset={offset}&length={length}&dtd={dtd}"
    print(final_url)
    
    
    html = requests.get(final_url).text
    soup = BeautifulSoup(html, 'html.parser')
    tables = soup('table')

    ### 1) 청약 및 배정현황 table  ## 최초 우리사주 배정량 / 최종 배정량
    table_1 = list(
        filter(lambda x: "우리사주조합" in x.text, tables))[0]
    info['init_employee_qty'] = table_1.select('tr')[2].select('td')[1].text
    info['final_employee_qty'] = table_1.select('tr')[2].select('td')[8].text


    ### 2) 최종 기관 의무보유확약기간 table -> 여기서 유통주식수 역추적
    ### 유통가능주식수 = 일반투자자 최종배정물량 + 기관투자자 최종배정물량 중 미확약
    table_2 = list(
        filter(lambda x: "확약기간" in x.text, tables))[0]
        
    print(len(table_2))


    info['stock_type'] = tables[1].select('td')[0].text ## 증권의 종류 : 기명식보통주
parse_issueReport(rcpNo2)

https://dart.fss.or.kr/report/viewer.do?rcpNo=20220630000860&dcmNo=8719784&eleId=5&offset=6117&length=97551&dtd=dart3.xsd
OrderedDict([('init_employee_qty', '60,000'), ('final_employee_qty', '60,000')])
7
